In [1]:
import os, sys, subprocess
# Colab setup ------------------
if "google.colab" in sys.modules:
    
    # Mount drive
    from google.colab import drive
    print('Select your Caltech Google Account')
    drive.mount('/content/drive/')

Select your Caltech Google Account
Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd

from scipy.signal import filtfilt, butter

import holoviews as hv
import panel as pn
# import holoviews.operation.datashader
import bokeh
from holoviews import opts
hv.extension('bokeh', 'matplotlib')

Output hidden; open in https://colab.research.google.com to view.

In [15]:
!pip install -e drive/MyDrive/My\ Science\ Practice/13\ Dissemination\ Publication\ Outreach/ERGo/erg

Obtaining file:///content/drive/MyDrive/My%20Science%20Practice/13%20Dissemination%20Publication%20Outreach/ERGo/erg
  Running setup.py develop for erg


In [18]:
import drive/MyDrive/My\ Science\ Practice/13\ Dissemination\ Publication\ Outreach/ERGo/erg

SyntaxError: ignored

In [ ]:
ergram = ERGio.ERG(r'../../data/BYB_Recording_2020-12-23_13.47.39.wav')

### Data validation: confirming that the shifted times are sawtoothed and the normal times look like a slippery staircase

In [ ]:
%%opts Curve {+axiswise}

hv.Curve(
    ergram.df['shifted time (s)'].values, 
    label='shifted'
) + hv.Curve(
    ergram.df['time (s)'].values,
    label='normal'
)

:Layout
   .Curve.Shifted :Curve   [x]   (y)
   .Curve.Normal  :Curve   [x]   (y)

### Visualize TTL pulses for each frequency

For a specific frequency, this is where all of that frequency occurred for each color.

In [ ]:
color_dict = {
    'IR':'deeppink',
    'R':'firebrick',
    'G':'green',
    'B':'blue',
    'UV':'violet'
}

In [ ]:
frequency_picker = pn.widgets.Select(
    name="Frequency (Hz)",
    options=sorted(list(ergram.df['theoretical frequency'].unique())),
    value=8,
    width=100
)

@pn.depends(frequency=frequency_picker.param.value)
def plot_stimuli_select_frequency(frequency):
    return hv.NdOverlay(
        {
            color: hv.Curve(
                data=ergram.df[
                    (ergram.df.color==color) 
                    & (ergram.df['theoretical frequency']==frequency)
                ],
                kdims=['time (s)'],
                vdims=[
                    'TTL'
                ],
            ).opts(
                color=color_dict[color],
                width=600,
                height=200
            ) for color in list(color_dict.keys())
        },
        kdims='Color'
    ).opts(
        legend_position='right'
    )



color_picker = pn.widgets.Select(
    name="Color",
    options=sorted(list(ergram.df.color.unique())),
    value='R',
    width=100
)

@pn.depends(color=color_picker.param.value)
def plot_stimuli_select_color(color):
    return hv.NdOverlay(
        {
            frequency: hv.Curve(
                data=ergram.df[
                    (ergram.df.color==color) 
                    & (ergram.df['theoretical frequency']==frequency)
                ],
                kdims=['time (s)'],
                vdims=['TTL'],
            ).opts(
                color=color_dict[color],
                width=600,
                height=200
            ) for frequency in list(ergram.df['theoretical frequency'].unique())
        },
        kdims='Color'
    ).opts(
        legend_position='right'
    )

pn.Column(frequency_picker, plot_stimuli_select_frequency, color_picker, plot_stimuli_select_color)


AbbreviatedException: AttributeError: 'str' object has no attribute 'dataspecs'

To view the original traceback, catch this exception and call print_traceback() method.

Column
    [0] Select(name='Frequency (Hz)', options=[1, 2, 4, 8, 16, ...], value=8, width=100)
    [1] ParamFunction(function)
    [2] Select(name='Color', options=['B', 'G', 'IR', ...], value='R', width=100)
    [3] ParamFunction(function)

### Visualize responses to light

In [ ]:

# Make a selector for the color to display
color_selector = pn.widgets.Select(
    name="Color",
    options=list(color_dict.keys()),
    value="R",
    width=100
)

# Plot the colors
@pn.depends(color=color_selector.param.value)
def plot_responses(color):
    return hv.Curve(
        data=ergram.df[ergram.df['color']==color],
        kdims=['shifted time (s)'],
        vdims=[
            ('channel 1', 'Voltage'),
            ('theoretical frequency', 'frequency (Hz)')
        ]
    ).groupby(
        ['theoretical frequency']
    ).layout(
        'theoretical frequency'
    ).opts(
        opts.Curve(
            line_width=2,
            color=color_dict[color],
            xaxis=None, yaxis=None
        )
    ).cols(5)

pn.Column(color_selector, plot_responses)

AbbreviatedException: AttributeError: 'str' object has no attribute 'dataspecs'

To view the original traceback, catch this exception and call print_traceback() method.

Column
    [0] Select(name='Color', options=['IR', 'R', 'G', ...], value='R', width=100)
    [1] ParamFunction(function)

So this is kind of interesting. Toward the end of the end of stimulation, especially with prolonged periods of stimulation (e.g., at high frequencies or low frequencies), you get some movement artifact. I saw the cockroaches really seem to not like that- they would squirm a lot. Another observation which I think is even more important, at around the halfway point, you see some short upward notch.

In [ ]:
def RMS(arr):
    # mean-center
    arr -= np.mean(arr)
    
    # Square
    squared = arr ** 2
    
    # Mean
    mean_squared = np.mean(squared)
    
    # Root
    root_mean_squared = np.sqrt(mean_squared)
    
    return root_mean_squared
    

In [ ]:
RMS_df = ergram.df.groupby(
    ['trial', 'color', 'theoretical frequency']
).agg(
    lambda x: RMS(x['channel 1'])
).reset_index()

KeyError: 'channel 1'

In [ ]:
RMS_df['power'] = RMS_df[0]

TODO: make color a numerical thing so you can make it the x-axid

In [ ]:
hv.Curve(
    data=RMS_df,
    kdims='theoretical frequency', # consider making this actual frequency
    vdims=['power','color'],
).groupby(
    'color'
).overlay('color')

NameError: name 'RMS_df' is not defined